In [7]:
import os
import uuid
import numpy as np
from flask import Flask, request, jsonify, render_template
import tensorflow as tf
from PIL import Image
from flask_cors import CORS
from jinja2 import TemplateNotFound 

# Path to the public folder
base_path = r'./temp/'

# Initialize Flask application
app = Flask(__name__, static_folder="public")
cors = CORS(app, origins=["http://localhost:3000"])

# Load the TensorFlow model
model_path = r"D:\Projects\Digit-recognizer-ML_Flask\best_model.keras"

@app.route('/process', methods=['POST'])
def upload_file():
    try:
        # Get the file from the POST request
        file = request.files['file']

        # Generate a UUID identifier
        c_uuid = str(uuid.uuid4())

        # Get the file extension
        file_ext = file.filename.rsplit('.', 1)[1].lower()

        # Construct the file path
        filename = os.path.join(base_path, c_uuid + '.' + file_ext)

        # Save the file
        file.save(filename)

        # Load the image and convert to grayscale
        img = Image.open(filename).convert("L")

        # Resize the image
        new_image = img.resize((28, 28))

        # Convert image to array and resize
        x = np.array(new_image).reshape((28, 28, 1))
        x = np.expand_dims(x, axis=0)
        images = np.vstack([x])

        # Predict the digit in the image
        classes = model.predict(images, batch_size=1)

        # Get the predicted digit (class)
        result = int(np.argmax(classes))
        print("Output:", result)
        # Close the image
        img.close()

        # Remove the uploaded image
        os.remove(filename)

        # Return the result as JSON
        return jsonify({'value': result})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Endpoint for digit recognition from uploaded image
@app.route('/', methods=['GET', 'POST'])
def start():
    try:
        return render_template(r'default.html')
    except TemplateNotFound:
        return "Template not found!", 404

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
    model = tf.keras.models.load_model(model_path)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.106:5000
Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


127.0.0.1 - - [03/Jul/2024 14:50:37] "POST /process HTTP/1.1" 200 -


Output: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


...

127.0.0.1 - - [03/Jul/2024 14:51:07] "POST /process HTTP/1.1" 200 -


Output: 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step


127.0.0.1 - - [03/Jul/2024 14:51:15] "POST /process HTTP/1.1" 200 -


Output: 5


TypeError: <class 'keras.src.models.sequential.Sequential'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras', 'class_name': 'Sequential', 'config': {'name': 'sequential', 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': [None, 28, 28, 1], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'zero_padding2d_1_input'}, 'registered_name': None}, {'module': 'keras.layers', 'class_name': 'ZeroPadding2D', 'config': {'name': 'zero_padding2d_1', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': [None, 28, 28, 1], 'padding': [[1, 1], [1, 1]], 'data_format': 'channels_last'}, 'registered_name': None, 'build_config': {'input_shape': [None, 28, 28, 1]}}, {'module': 'keras.layers', 'class_name': 'Conv2D', 'config': {'name': 'conv2d_2', 'trai